In [13]:
# Import Libraries
from keras.models import Sequential, clone_model, Model
from keras.layers import Dense, Dropout, Input
from keras.layers.merge import concatenate
from keras.datasets import mnist
from keras.utils import np_utils
from keras import backend as K
import matplotlib.pyplot as plt
import numpy as np

In [14]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(X_train.shape[0], -1)/255
X_test = X_test.reshape(X_test.shape[0], -1)/255
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(60000, 784)
(10000, 784)
(60000, 10)
(10000, 10)


In [34]:
def baseline_model(num_pixels, num_classes):
    inpt1 = Input(shape=(num_pixels,))
    dense1 = Dense(num_pixels,kernel_initializer='normal',activation='relu')(inpt1)
    
    inpt2 = Input(shape=(num_pixels,))
    dense2 = Dense(num_pixels,kernel_initializer='normal',activation='relu')(inpt2)
    
    concat_layer = concatenate([dense1, dense2], axis=-1)
    logits = Dense(num_classes, kernel_initializer='normal',activation='softmax')(concat_layer)
    model = Model([inpt1, inpt2], logits)
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

In [36]:
num_pixels = X_train.shape[1]
num_classes = y_train.shape[1]

model = baseline_model(num_pixels=num_pixels, num_classes=num_classes)
model.fit(x=[X_train, X_train], y=y_train, validation_split=0.1, epochs=15, batch_size=200, verbose=1)

Train on 54000 samples, validate on 6000 samples
Epoch 1/15
54000/54000 [==============================] - 4s 73us/step - loss: 0.2503 - acc: 0.9273 - val_loss: 0.1044 - val_acc: 0.9708
Epoch 2/15
54000/54000 [==============================] - 3s 50us/step - loss: 0.0938 - acc: 0.9730 - val_loss: 0.0822 - val_acc: 0.9768
Epoch 3/15
54000/54000 [==============================] - 3s 50us/step - loss: 0.0574 - acc: 0.9831 - val_loss: 0.0695 - val_acc: 0.9802
Epoch 4/15
54000/54000 [==============================] - 3s 49us/step - loss: 0.0376 - acc: 0.9895 - val_loss: 0.0630 - val_acc: 0.9808
Epoch 5/15
54000/54000 [==============================] - 3s 49us/step - loss: 0.0265 - acc: 0.9930 - val_loss: 0.0604 - val_acc: 0.9823
Epoch 6/15
54000/54000 [==============================] - 3s 50us/step - loss: 0.0180 - acc: 0.9954 - val_loss: 0.0663 - val_acc: 0.9798
Epoch 7/15
54000/54000 [==============================] - 3s 50us/step - loss: 0.0127 - acc: 0.9969 - val_loss: 0.0567 - val_acc:

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 784)               615440    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                7850      
Total params: 623,290
Trainable params: 623,290
Non-trainable params: 0
_________________________________________________________________


In [67]:
def generator_model(input_dimension, hidden_size, discriminator):
    inpt1 = Input(shape=input_dimension)
    l = inpt1
    for hid in hidden_size:
        l = Dense(units=hid, kernel_initializer='normal',activation='relu')(l)
    model_d = clone_model(model=discriminator)
    for layer in model_d.layers:
        layer.trainable = False
        
    inpt2 = Input(shape=(784,))
    out = model_d([l, inpt2])
    model = Model([inpt1, inpt2], out)
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

In [68]:
model_G = generator_model((10,), [32, 784], model)
model_G.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_35 (InputLayer)           (None, 10)           0                                            
__________________________________________________________________________________________________
dense_50 (Dense)                (None, 32)           352         input_35[0][0]                   
__________________________________________________________________________________________________
dense_51 (Dense)                (None, 784)          25872       dense_50[0][0]                   
__________________________________________________________________________________________________
input_36 (InputLayer)           (None, 784)          0                                            
__________________________________________________________________________________________________
model_5 (M

In [62]:
temp_x = np.random.uniform(high=1,low=0, size=(10000, 10))
y = np.random.choice(range(10), size=10000)
y = np_utils.to_categorical(y)
model_G.fit(x=[temp_x, ], y=y, epochs=10)

Epoch 1/10
10000/10000 [==============================] - 1s 118us/step - loss: 2.3030 - acc: 0.0997
Epoch 2/10
10000/10000 [==============================] - 1s 104us/step - loss: 2.3025 - acc: 0.1077
Epoch 3/10
10000/10000 [==============================] - 1s 105us/step - loss: 2.3026 - acc: 0.1086
Epoch 4/10
10000/10000 [==============================] - 1s 104us/step - loss: 2.3020 - acc: 0.1060
Epoch 5/10
10000/10000 [==============================] - 1s 104us/step - loss: 2.3023 - acc: 0.1063
Epoch 6/10
10000/10000 [==============================] - 1s 105us/step - loss: 2.3015 - acc: 0.1087
Epoch 7/10
10000/10000 [==============================] - 1s 104us/step - loss: 2.3009 - acc: 0.1092
Epoch 8/10
10000/10000 [==============================] - 1s 104us/step - loss: 2.3004 - acc: 0.1091
Epoch 9/10
10000/10000 [==============================] - 1s 104us/step - loss: 2.2993 - acc: 0.1167
Epoch 10/10
10000/10000 [==============================] - 1s 105us/step - loss: 2.2986 - a

In [63]:
model_G.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_32 (InputLayer)           (None, 10)           0                                            
__________________________________________________________________________________________________
dense_46 (Dense)                (None, 32)           352         input_32[0][0]                   
__________________________________________________________________________________________________
dense_47 (Dense)                (None, 784)          25872       dense_46[0][0]                   
__________________________________________________________________________________________________
model_5 (Model)                 (None, 10)           1246570     dense_47[0][0]                   
                                                                 dense_47[0][0]                   
Total para